In [175]:
import numpy as np
import pandas as pd
from scipy import stats as st

DATASET_PATH = '../input/analytics_session/summer_camp_data.csv'

df = pd.read_csv(DATASET_PATH)

### a. Какой рекламный канал принёс больше всего дохода за всё время?

In [176]:
sort_df = df.sort_values(by='revenue', ascending=False)
print(sort_df.iloc[0])

date                     2020-09-11
source                     (direct)
medium                       (none)
delivery_available    Не определено
device_type           Не определено
promo_activated                  no
filter_used                      no
pageviews                       641
visits                         1329
productClick                      0
addToCart                         0
checkout                          0
transactions                   1005
revenue                   1397420.0
Name: 17665, dtype: object


### b. Как изменился средний чек транзакции после введения коронавирусных ограничений? Как он изменился для пользователей, покупающих с промокодом и без? (параметр «promo_activated»)

In [177]:
# выделим два датафрейма, первый будет содержать записи до ковидных 
# ограничений, второй после
before_cov = df['date'] <= '2020-03-16'
before_cov_df = df.loc[before_cov]

after_cov = df['date'] > '2020-03-16'
after_cov_df = df.loc[after_cov]

# рассчитываем средний чек до и после ограничений
mean_trans_before_cov = before_cov_df['transactions'].mean()
mean_trans_after_cov = after_cov_df['transactions'].mean()

print(f'Средний чек до введения ограничений: {mean_trans_before_cov:.2f}')
print(f'Средний чек после введения ограничений: {mean_trans_after_cov:.2f}')
print(f'Средний чек снизился на {100 - mean_trans_after_cov / mean_trans_before_cov * 100:.2f} процентов после введения коронавирусных ограничений')

# выделим пользователей с промокодом и без
before_cov_activ_df = before_cov_df.loc[before_cov_df['promo_activated'] == 'yes']
before_cov_not_activ_df = before_cov_df.loc[before_cov_df['promo_activated'] == 'no']
after_cov_activ_df = after_cov_df.loc[after_cov_df['promo_activated'] == 'yes']
after_cov_not_activ_df = after_cov_df.loc[after_cov_df['promo_activated'] == 'no']

# рассчитаем средние показатели
mean_before_cov_activ = before_cov_activ_df['transactions'].mean()
mean_before_cov_not_activ = before_cov_not_activ_df['transactions'].mean()
mean_after_cov_activ = after_cov_activ_df['transactions'].mean()
mean_after_cov_not_activ = after_cov_not_activ_df['transactions'].mean()

print("////////////////")

print("Средние чеки пользователей до введения ковидных ограничений:")
print(f'с промокодом: {mean_before_cov_activ:.2f}')
print(f'без промокода: {mean_before_cov_not_activ:.2f}')

print("Средние чеки пользователей после введения ковидных ограничений:")
print(f'с промокодом: {mean_after_cov_activ:.2f}')
print(f'без промокода: {mean_after_cov_not_activ:.2f}')

print("////////////////")
print("Средний чек пользователя, покупающего с промокодом выше, чем чек пользователя, осуществляющего покупки без промокода")

Средний чек до введения ограничений: 21.87
Средний чек после введения ограничений: 19.47
Средний чек снизился на 10.98 процентов после введения коронавирусных ограничений
////////////////
Средние чеки пользователей до введения ковидных ограничений:
с промокодом: 23.27
без промокода: 20.69
Средние чеки пользователей после введения ковидных ограничений:
с промокодом: 21.19
без промокода: 17.85
////////////////
Средний чек пользователя, покупающего с промокодом выше, чем чек пользователя, осуществляющего покупки без промокода


### c. Можно ли с уверенностью в 95% сказать, что CR (коэффициент конверсии в транзакцию) в выходные дни отличается от CR в будние дни?

In [178]:
# добавим столбцы с днями недели
df['date'] = pd.to_datetime(df['date'])
df['weekday'] = df['date'].dt.weekday

workdays_df = df.query('weekday ! = 5 and weekday ! = 6')
rest_days_df = df.query('weekday == 5 or weekday == 6')

# считаем CR
workdays_df['cr'] = workdays_df['transactions'] / workdays_df['visits'] * 100
rest_days_df['cr'] = rest_days_df['transactions'] / rest_days_df['visits'] * 100

# считаем конверсию в будние и выходные дни
mean_cr_workday = workdays_df['cr'].mean()
mean_cr_rest = rest_days_df['cr'].mean()

print(f'Средняя конверсия в рабочие дни: {mean_cr_workday:.1f}%')
print(f'Средняя конверсия в выходные дни: {mean_cr_rest:.1f}%')


Средняя конверсия в рабочие дни: 37.6%
Средняя конверсия в выходные дни: 40.0%


/tmp/ipykernel_216712/3397309698.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  workdays_df['cr'] = workdays_df['transactions'] / workdays_df['visits'] * 100
/tmp/ipykernel_216712/3397309698.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rest_days_df['cr'] = rest_days_df['transactions'] / rest_days_df['visits'] * 100


H0 - CR в выходные дни не отличается от CR в будние

H1 - CR в выходные дни отличается от CR в будние

In [180]:
# вычислим дисперсию CR
var_workdays = np.var(workdays_df['cr'], ddof=1)
var_rest_days = np.var(rest_days_df['cr'], ddof=1)

print(var_workdays)
print(var_rest_days)

# критерий статистической значимости
alpha = 0.5

1489.6795685833022
1456.1746298912371


In [184]:
test = st.ttest_ind(workdays_df['cr'],
                   rest_days_df['cr'], equal_var = False)

print(f'p-value: {test.pvalue}')

if (test.pvalue < alpha):
    print("Отвергаем нулевую гипотезу")
else:
    print("Не получилось отвергнуть нулевую гипотезу") 


p-value: 7.476651926325496e-11
Отвергаем нулевую гипотезу


Можно сделать вывод, что с уверенностью в 95% CR в выходные дни отличается от CR в будние.

### d. Вам необходимо спрогнозировать объем дохода, полученного с пользователей, приведенных на сайт контекстной рекламой (medium = cpc) на полгода вперед. Опишите, как бы вы подошли к этой задаче и какие дополнительные данные вам понадобятся?

### План:
1. Произвести ресемплирование данных.
2. Проанализировать данные, произвести графический анализ временных рядов.
3. Определить тренд и сезонность.
4. Выделить тренировочную и тестовую выборку.
5. Обучить модели, оценить качество, выбрать наилучшую.
6. Проверить модель на тестовой выборке.
7. Сделать выводы.


In [193]:
temp_data = pd.read_csv(DATASET_PATH, sep=',', index_col=[0], parse_dates=[0])
data = temp_data.query("medium == 'cpc'")